# Training a single neuron

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Import Dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

## Data Processing

In [5]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [6]:
df.shape

(569, 31)

## Train Test Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [9]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


## Scaling 

In [10]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
X_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

In [12]:
y_train[:10]

68     B
181    M
63     B
248    B
60     B
15     M
290    B
137    B
155    B
517    M
Name: diagnosis, dtype: object

## Label Encoding : Traget

In [13]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [14]:
y_train[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [17]:
X_train.shape

torch.Size([455, 30])

## Numpy array to Pytorch Tensors

In [15]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [16]:
type(X_test)

torch.Tensor

## Define Model

In [26]:
class MySimpleNN():

    def __init__(self, X):

        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)

    # forward pass
    def forward(self, X):
        # calculate z
        z = torch.matmul(X, self.weights) + self.bias
        # calculate activation function
        y_pred = torch.sigmoid(z)
        return y_pred

    # loss calculation
    def binary_cross_entropy(self, y_pred, y):
        # To avoid log(0), clamp predicted values to a small epsilon range
        epsilon = 1e-15
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
        
        # Binary cross-entropy formula
        loss = -(y * torch.log(y_pred) + (1 - y) * torch.log(1 - y_pred)).mean()
        return loss

    

## Important Parameters

In [19]:
learning_rate = 0.1
epochs = 25

## Training Pipeline

In [28]:
# create model
model = MySimpleNN(X_train)

# define loop
for epoch in range(epochs):
    
    # forward pass
    y_pred = model.forward(X_train)

    # loss calculation
    loss = model.binary_cross_entropy(y_pred, y_train)

    # backward pass
    loss.backward()

    # parameter update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    # zero grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()

    # print loss in each epoch
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 3.731259923307155
Epoch 2, Loss: 3.520386423849176
Epoch 3, Loss: 3.3109730850345365
Epoch 4, Loss: 3.1032481801116916
Epoch 5, Loss: 2.8975806190460744
Epoch 6, Loss: 2.6944273432515113
Epoch 7, Loss: 2.494359292685886
Epoch 8, Loss: 2.298091967225282
Epoch 9, Loss: 2.1065281923642876
Epoch 10, Loss: 1.9208192577803678
Epoch 11, Loss: 1.7424400805942064
Epoch 12, Loss: 1.5732620559446195
Epoch 13, Loss: 1.4155900153438676
Epoch 14, Loss: 1.272108794590968
Epoch 15, Loss: 1.1456679827498106
Epoch 16, Loss: 1.0388303466118551
Epoch 17, Loss: 0.9531735829172576
Epoch 18, Loss: 0.8885758372596183
Epoch 19, Loss: 0.8427961887701277
Epoch 20, Loss: 0.8117441994031566
Epoch 21, Loss: 0.7908719433992691
Epoch 22, Loss: 0.7765182012326944
Epoch 23, Loss: 0.7662090880503497
Epoch 24, Loss: 0.758402042155897
Epoch 25, Loss: 0.7521690222209603


In [22]:
print("Weights shape", model.weights.shape)
print("Bias shape", model.bias.shape)

Weights shape torch.Size([30, 1])
Bias shape torch.Size([1])


In [29]:
print(model.weights)

tensor([[-0.0569],
        [ 0.4490],
        [ 0.2083],
        [ 0.5491],
        [-0.0502],
        [ 0.0670],
        [ 0.1306],
        [ 0.0998],
        [-0.3085],
        [ 0.4533],
        [-0.1568],
        [ 0.2388],
        [-0.3795],
        [-0.0749],
        [ 0.4694],
        [ 0.0098],
        [-0.3548],
        [ 0.1054],
        [ 0.1269],
        [-0.1407],
        [ 0.1037],
        [-0.0914],
        [-0.2059],
        [-0.0867],
        [-0.0972],
        [ 0.0446],
        [-0.4142],
        [ 0.1925],
        [ 0.0864],
        [ 0.0713]], dtype=torch.float64, requires_grad=True)


## Evaluation

In [31]:
with torch.no_grad():
    y_pred = model.forward(X_test)

y_pred

tensor([[0.3449],
        [0.3587],
        [0.2811],
        [0.5913],
        [0.5683],
        [0.6380],
        [0.4138],
        [0.3385],
        [0.4423],
        [0.2326],
        [0.5241],
        [0.3776],
        [0.4431],
        [0.7149],
        [0.2057],
        [0.9500],
        [0.3317],
        [0.7849],
        [0.5716],
        [0.5130],
        [0.3845],
        [0.4507],
        [0.6113],
        [0.5465],
        [0.6078],
        [0.4254],
        [0.1761],
        [0.8197],
        [0.4766],
        [0.4346],
        [0.3910],
        [0.2813],
        [0.5043],
        [0.3312],
        [0.5483],
        [0.4432],
        [0.5310],
        [0.3230],
        [0.3855],
        [0.5660],
        [0.3652],
        [0.6637],
        [0.3082],
        [0.3832],
        [0.6488],
        [0.5510],
        [0.7243],
        [0.8033],
        [0.3248],
        [0.3046],
        [0.3437],
        [0.3728],
        [0.7120],
        [0.4860],
        [0.5407],
        [0

In [32]:
# convert in 1 or 0
y_pred = (y_pred > 0.5).float()

In [33]:
y_pred

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
      

## Accuracy

In [34]:
accuracy = (y_pred == y_test).float().mean()

print(accuracy)

tensor(0.5086)
